# Road Following - Live demo(デモ)

このノートブックでは、トラックの上をスムーズにJetBotが移動させるために学習済みモデルを使用します。

### Load Trained Model(学習済みモデルの読み込み)

We will assume that you have already downloaded ``best_steering_model_xy.pth`` to work station as instructed in "train_model.ipynb" notebook. Now, you should upload model file to JetBot in to this notebooks's directory. Once that's finished there should be a file named ``best_steering_model_xy.pth`` in this notebook's directory.


> Please make sure the file has uploaded fully before calling the next cell

PyTorchモデルを初期化するために下記コードを実行します。これはトレーニング用のnotebookでもおなじみですね。

In [ ]:
import torchvision
import torch


model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)

次に、TensorRTに対応した``best_steering_model_xy_trt.pth``から学習の重みづけをロードします。

In [ ]:
model.load_state_dict(torch.load('best_steering_model_xy.pth'))

現在、モデルの重みはCPUメモリにあり、以下のコードを実行してGPUデバイスに転送します。

In [ ]:
device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

### Creating the Pre-Processing Function

モデルを読み込みましたが、まだわずかな問題があります。学習済みモデルのフォーマットと、カメラの形式と完全に一致しません。これを解消するために、 いくつかの前処理を行う必要があります。これらは、下記の手順になります。

1. HWC layoutからCHW layoutに変換
2. トレーニング中に使ったのと同じパラメーターを使用して正規化します（カメラは[0、255]の範囲の値を提供し、ロードされた画像は[0、1]の範囲でトレーニングするため、255.0でスケーリングする必要があります
3. dataをCPUメモリからGPUメモリに転送します
4. バッチディメンションを追加する

In [ ]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

すばらしい、これでカメラ形式からneural networkのインプットのフォーマットに変換するための、pre-processing関数を定義する事ができました。　

カメラを起動し、画面に表示します。カメラの起動には、大分なれてきましたね。

In [ ]:
!echo jetbot | sudo -S systemctl restart nvargus-daemon

In [ ]:
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224, fps=2)

image_widget = ipywidgets.Image()

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

モーターの制御に必要なrobotインスタンスを生成します。

In [ ]:
from jetbot import Robot

robot = Robot()

JetBotを制御するためにsliderを制御します。
> メモ: 最もよく知られている構成のために、スライダー値を初期化していますが、あなたのデータセットでは、うまく機能しない可能性があるります。そのため、環境に応じてスライダーを増減してセットアップしてください。


1. スピードコントローラー(speed_gain_slider): ``speed_gain_slider``の値を増やすとJetBotがスタートします。
2. ステアリングゲインコントローラー (steering_gain_sloder):JetBotが左右にブレて不安定な場合は、スムーズに走るようになるまで、`` steering_gain_slider``の値を減らす必要があります
3. バイアスコントローラー(steering_bias_slider): JetBotがトラックの右端または左端に偏って走行する場合は、JetBotが中央のラインまたはトラックをたどるまでこのスライダーの値を調整します。ここでは、モーターバイアスと同様にカメラのoffsetも考慮します。


> 注：JetBotの道路追従動作をスムーズにする必要するために、上記のスライダーを低速で移動して調整します。

In [ ]:
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

次に、JetBotが何を考えているかを表示するためにいくつかのsliderを画面に表示します。x, y sliderは、推論されたx, yの値が表示されます。

ステアリングsliderは、推定ステアリング値が表示されます。忘れないでください、この値はターゲットの実際の角度ではなく、ほぼ比例した単純な値です。実際の角度が``0``の場合は、これはゼロを意味し、実際の角度で増減します。

In [ ]:
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

次は、カメラの画像が変わるたびに呼び出される関数を生成します。この関数は、下記ステップを実行します。

1. カメラのイメージのPre-process
2. neural networkの実行
3. ステアリング値を計算する
4. 比例/微分制御（PD）を使用してモーターを制御する

In [ ]:
angle = 0.0
angle_last = 0.0

def execute(change):
    global angle, angle_last
    image = change['new']
    xy = model(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    speed_slider.value = speed_gain_slider.value
    
    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    
    robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
execute({'new': camera.value})

素晴らしい! neural network実行関数が作られましたが、カメラをプロセスに紐付けます。

observe関数でそれを実現します。

> 注意: このコードでロボットが動きだします。十分なスペースを確保してください。collision avoidanceが動くでしょう、neral networkは、訓練されたデータと同じくらいに動くはずです。

``speed gain``を0から少しずつ値を大きくし、全しさせます。
``steering gain``でハンドルの切れ角を調整します。0に近いほど、切れ角がゆるくなります。

In [ ]:
import ipywidgets
import time

model_start_button = ipywidgets.Button(description='start jetbot')
model_stop_button = ipywidgets.Button(description='stop jetbot')

def start_model(c):
    camera.observe(execute, names='value')
model_start_button.on_click(start_model)
    
def stop_model(c):
    camera.unobserve(execute, names='value')
    time.sleep(1)
    robot.stop()
model_stop_button.on_click(stop_model)

model_widget = ipywidgets.VBox([
    ipywidgets.HBox([speed_gain_slider, steering_gain_slider]),
    ipywidgets.HBox([image_widget]),
    steering_slider,
    ipywidgets.HBox([model_start_button, model_stop_button])
])

display(model_widget)

驚くばかり！ロボットが接続されている場合、新しいカメラフレームごとに新しいコマンドが生成されるはずです。

あなたがデータセットを作ったLegoかトラックのの上にJetBotを置いてください、そしてトラックをフォローするかどうか確認してください。

この挙動を止めたい場合は、下記コードを実行する事で、紐付けをやめれます。

In [ ]:
#import time

#camera.unobserve(execute, names='value')

#time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

#robot.stop()

追記) うまくモーターが止まらない場合は、Robotクラスを再度初期化する

In [ ]:
#robot = Robot()
#robot.stop()

### Conclusion
以上がLive demoです。うまく行けば、ロボットが道路をスムーズにしたのではないでしょうか？　

avoiding collisionsが上手く行かない場合、失敗した箇所を見つけてください。美しさは、これら上手くいかないシナリオについてより多くのデータを収集すれば、ロボットはさらに良くなるはずです